In [ ]:
!pip install PyDrive

In [1]:
import os
import mykeys

project_name = 'CH13_LCEL'

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["LANGCHAIN_API_KEY"] = mykeys.get_key('LANG')
os.environ["LANGCHAIN_HUB_API_KEY"] = mykeys.get_key('LANG')
os.environ["OPENAI_API_KEY"] = mykeys.get_key('GPT')
os.environ["GOOGLE_API_KEY"] = mykeys.get_key('GOO')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = mykeys.get_key('HF')
os.environ["UPSTAGE_API_KEY"] = mykeys.get_key('UP')
os.environ["COHERE_API_KEY"] = mykeys.get_key('COH')
os.environ["JINA_API_KEY"] = mykeys.get_key('JINA')
os.environ["ANTHROPIC_API_KEY"] = mykeys.get_key('ANT')

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.


In [3]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith(project_name)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH13_LCEL


In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith(project_name, set_enable=False)

CH13 LangChain Expression Language(LCEL)

RunnablePassthrough

In [4]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    # 전달된 입력을 그대로 반환하는 Runnable을 설정합니다.
    passed=RunnablePassthrough(),
    # 입력의 "num" 값에 3을 곱한 결과를 반환하는 Runnable을 설정합니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 입력의 "num" 값에 1을 더한 결과를 반환하는 Runnable을 설정합니다.
    modified=lambda x: x["num"] + 1,
)

# {"num": 1}을 입력으로 Runnable을 실행합니다.
runnable.invoke({"num": 1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [5]:
r = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)
r.invoke({"num": 1})


{'num': 1, 'mult': 3}

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 텍스트로부터 FAISS 벡터 저장소를 생성합니다.
vectorstore = FAISS.from_texts(
    [
        "테디는 랭체인 주식회사에서 근무를 하였습니다.",
        "셜리는 테디와 같은 회사에서 근무하였습니다.",
        "테디의 직업은 개발자입니다.",
        "셜리의 직업은 디자이너입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)
# 벡터 저장소를 검색기로 사용합니다.
retriever = vectorstore.as_retriever()
# 템플릿을 정의합니다.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
# 템플릿으로부터 채팅 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1")


# 문서를 포맷팅하는 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# 검색 체인을 구성합니다.
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [10]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("테디의 직업은 무엇입니까?")


'테디의 직업은 개발자입니다.'

In [11]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("셜리의 직업은 무엇입니까?")


'셜리의 직업은 디자이너입니다.'

In [ ]:
Runnables 구조 검토

In [12]:
!pip install -qU faiss-cpu tiktoken

# 그래프를 그리기 위한 라이브러리 설치
!pip install -qU grandalf


In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성합니다.
    ["Teddy is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성합니다.
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}  

Question: {question}"""

prompt = ChatPromptTemplate.from_template(
    template
)  # 템플릿을 기반으로 ChatPromptTemplate을 생성합니다.

model = ChatOpenAI(model="gpt-4.1")  # ChatOpenAI 모델을 초기화합니다.

# chain 을 생성합니다.
chain = (
    # 검색 컨텍스트와 질문을 지정합니다.
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt  # 프롬프트를 생성합니다.
    | model  # 언어 모델을 실행합니다.
    | StrOutputParser()  # 출력 결과를 문자열로 파싱합니다.
)


In [14]:
# 체인의 그래프에서 노드를 가져옵니다.
chain.get_graph().nodes


{'d968fce6edcf43459110060f7428e629': Node(id='d968fce6edcf43459110060f7428e629', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None),
 'fe581b0ab47f419ba0fefdb06edbebcd': Node(id='fe581b0ab47f419ba0fefdb06edbebcd', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None),
 'c2429b10f75d4ccf89289198fdf74590': Node(id='c2429b10f75d4ccf89289198fdf74590', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11a8f2690>, search_kwargs={}), metadata=None),
 '1999a9c6380440d59e542ed3a6f28989': Node(id='1999a9c6380440d59e542ed3a6f28989', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 '77355f87ee9f499699fb3fc511175879': Node(id='77355f87ee9f499699fb3fc511175879', name='ChatPromptTemplate', data=ChatP

In [15]:
# 체인의 그래프에서 엣지를 가져옵니다.
chain.get_graph().edges


[Edge(source='e9a99275bfad46bc8ae6e92ef05aa575', target='238c1241dec9442abc14bdfe7e92442f', data=None, conditional=False),
 Edge(source='238c1241dec9442abc14bdfe7e92442f', target='b4021e7636274100ba819a24db4812fd', data=None, conditional=False),
 Edge(source='e9a99275bfad46bc8ae6e92ef05aa575', target='950207c1873948ff80295e6364dfbef2', data=None, conditional=False),
 Edge(source='950207c1873948ff80295e6364dfbef2', target='b4021e7636274100ba819a24db4812fd', data=None, conditional=False),
 Edge(source='b4021e7636274100ba819a24db4812fd', target='7c033d7cada942a7b7d2d49cd3ba25c6', data=None, conditional=False),
 Edge(source='7c033d7cada942a7b7d2d49cd3ba25c6', target='980b3268b6744788bdcfb0cbe27ed2e0', data=None, conditional=False),
 Edge(source='0760c592df55435f8991d49e76107f4d', target='3e20e6dfb79546639864db238bbfbda4', data=None, conditional=False),
 Edge(source='980b3268b6744788bdcfb0cbe27ed2e0', target='0760c592df55435f8991d49e76107f4d', data=None, conditional=False)]

In [16]:
# 체인의 그래프를 ASCII 형식으로 출력합니다.
chain.get_graph().print_ascii()


           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

In [17]:
chain.get_prompts()  # 체인에서 사용되는 프롬프트를 가져옵니다.


[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}  \n\nQuestion: {question}'), additional_kwargs={})])]

RunnableLambda

In [18]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


def length_function(text):  # 텍스트의 길이를 반환하는 함수
    return len(text)


def _multiple_length_function(text1, text2):  # 두 텍스트의 길이를 곱하는 함수
    return len(text1) * len(text2)


def multiple_length_function(  # 2개 인자를 받는 함수로 연결하는 wrapper 함수
    _dict,
):  # 딕셔너리에서 "text1"과 "text2"의 길이를 곱하는 함수
    return _multiple_length_function(_dict["text1"], _dict["text2"])


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")
# ChatOpenAI 모델 초기화
model = ChatOpenAI()

# 프롬프트와 모델을 연결하여 체인 생성
chain1 = prompt | model

# 체인 구성
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()
)


In [19]:
# 주어진 인자들로 체인을 실행합니다.
chain.invoke({"input_1": "bar", "input_2": "gah"})


'3 + 9 is equal to 12.'

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
import json


def parse_or_fix(text: str, config: RunnableConfig):
    # 다음 텍스트를 수정하는 프롬프트 템플릿을 생성합니다.
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\ntext\n{input}\n\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    # 최대 3번 시도합니다.
    for _ in range(3):
        try:
            # JSON 형식으로 텍스트를 파싱합니다.
            return json.loads(text)
        except Exception as e:
            # 파싱 중 오류가 발생하면 수정 체인을 호출하여 텍스트를 수정합니다.
            text = fixing_chain.invoke({"input": text, "error": e}, config)
            print(f"config: {config}")
    # 파싱에 실패한 경우 "Failed to parse" 문자열을 반환합니다.
    return "Failed to parse"


In [21]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    # RunnableLambda를 사용하여 parse_or_fix 함수를 호출합니다.
    output = RunnableLambda(parse_or_fix).invoke(
        input="{foo:: bar}",
        config={"tags": ["my-tag"], "callbacks": [cb]},  # config 를 전달합니다.
    )
    # 수정한 결과를 출력합니다.
    print(f"\n\n수정한결과:\n{output}")


config: {'tags': ['my-tag'], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x10e909110>, 'recursion_limit': 25, 'configurable': {}}


수정한결과:
{'foo': 'bar'}


In [22]:
output


{'foo': 'bar'}

LLM 체인 라우팅(RunnableLambda, RunnableBranch)

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

# 체인을 생성합니다.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4.1")
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)


In [25]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})


'수학'

In [26]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})


'과학'

In [27]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "Google은 어떤 회사인가요?"})


'기타'

In [28]:
model_id = 'gpt-4.1'
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model=model_id)
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model=model_id)
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model=model_id)
)


In [29]:
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # datascience_chain을 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain


In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)


In [31]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})


'깨봉선생님께서 말씀하시기를, 미적분이란 수학에서 변화와 누적이라는 두 가지 중요한 개념을 다루는 분야입니다. 먼저, 미분은 어떤 함수가 아주 작은 양만큼 변할 때 그 함수값이 얼마나 변하는지, 즉 변화의 비율을 알아내는 방법입니다. 예를 들어, 자동차의 주행 거리와 시간의 관계에서 순간 속도를 구하는 것이 미분의 예입니다. 반대로 적분은 작은 변화들이 모여서 전체적으로 얼마나 쌓였는지, 즉 누적된 양을 구하는 방법입니다. 예를 들어, 속도가 주어졌을 때 전체 이동 거리를 구하는 것이 적분의 예입니다. 따라서 미적분은 자연 현상이나 여러 문제에서 연속적인 변화와 누적을 이해하고 계산하는 데 매우 중요한 도구라고 하셨습니다.'

In [32]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})


'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 모든 물체 사이에 작용하는 인력입니다. 뉴턴 선생님의 만유인력의 법칙에 따르면, 두 물체는 그 질량의 곱에 비례하고, 두 물체 사이의 거리의 제곱에 반비례하는 힘으로 서로를 끌어당깁니다. 이러한 중력의 작용 덕분에 사과가 나무에서 떨어지고, 지구가 태양 주위를 돌며, 달이 지구 주위를 도는 것입니다. 다시 말해, 중력은 우주의 모든 물체를 서로 연결하는 근본적인 힘입니다.'

In [33]:
# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})


'RAG(Retrieval Augmented Generation)은 외부 데이터베이스에서 관련 정보를 검색(Retrieval)한 후, 이를 바탕으로 텍스트를 생성(Generation)하는 자연어 처리 기술입니다. 즉, 검색과 생성 모델을 결합하여 더 정확하고 풍부한 답변을 만드는 방식입니다.'

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)


In [35]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})


'깨봉선생님께서 말씀하시기를, 미적분은 수학에서 변화와 누적을 다루는 중요한 분야입니다. 크게 미분과 적분으로 나눌 수 있는데요.  \n미분은 어떤 양이 아주 조금 변할 때 그에 따라 함수가 얼마나 변화하는지, 즉 변화율을 구하는 것이고, 적분은 어떤 양이 누적되어 전체적으로 얼마나 되는지, 즉 넓이나 부피를 구하는 과정입니다.  \n예를 들어 미분은 곡선의 순간 기울기를, 적분은 곡선 아래의 넓이나 도형의 부피를 구할 때 사용합니다. 미적분의 기본정리는 미분과 적분이 서로 깊은 관계가 있음을 보여줍니다.  \n즉, 미적분은 우리의 일상과 자연 현상에서 일어나는 연속적인 변화와 그 누적을 수학적으로 분석하는 핵심적인 도구라고 할 수 있습니다.'

In [36]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})


'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 지구와 같은 천체가 질량을 가진 물체를 자신 쪽으로 끌어당기는 힘으로 인해 나타나는 가속도라 하셨습니다. 중력 가속도 \\( g \\)는 다음의 공식으로 계산된다고 말씀하셨지요.\n\n\\[\ng = \\frac{GM}{r^2}\n\\]\n\n여기서  \n- \\( G \\)는 만유인력 상수(약 \\( 6.674 \\times 10^{-11} \\, \\text{N} \\cdot \\text{m}^2/\\text{kg}^2 \\)),  \n- \\( M \\)은 천체의 질량(예를 들어, 지구의 질량 약 \\( 5.97 \\times 10^{24} \\, \\text{kg} \\)),  \n- \\( r \\)은 천체 중심으로부터의 거리(지표면에서는 지구 반지름 약 \\( 6.371 \\times 10^6 \\, \\text{m} \\))입니다.\n\n따라서 지구 표면에서의 중력 가속도는 약 \\( 9.8 \\, \\text{m/s}^2 \\)로 계산됩니다. 이러한 값이 나오게 된 것은 만유인력 법칙과 운동 법칙(제2법칙)을 결합하여 얻어진 결과라 아이작 뉴턴 선생님께서 설명하셨습니다.'

In [37]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})


'RAG(Retrieval Augmented Generation)은 외부 지식 베이스에서 관련 정보를 검색(Retrieval)한 후, 이를 활용해 답변을 생성(Generation)하는 인공지능 모델 아키텍처입니다. 즉, 문서 등에서 정보를 찾아와 생성형 AI가 보다 정확하고 풍부한 응답을 할 수 있도록 합니다.'

RunnableParallel

In [38]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 텍스트로부터 FAISS 벡터 저장소를 생성합니다.
vectorstore = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"], embedding=OpenAIEmbeddings()
)
# 벡터 저장소를 검색기로 사용합니다.
retriever = vectorstore.as_retriever()
# 템플릿을 정의합니다.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
# 템플릿으로부터 채팅 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI(model="gpt-4.1")

# 검색 체인을 구성합니다.
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("What is Teddy's occupation?")


"Teddy's occupation is an AI engineer."

In [39]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 텍스트로부터 FAISS 벡터 저장소를 생성합니다.
vectorstore = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"], embedding=OpenAIEmbeddings()
)
# 벡터 저장소를 검색기로 사용합니다.
retriever = vectorstore.as_retriever()

# 템플릿을 정의합니다.
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
# 템플릿으로부터 채팅 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# 체인을 구성합니다.
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 체인을 호출하여 질문에 답변합니다.
chain.invoke({"question": "What is Teddy's occupation?", "language": "Korean"})


'Teddy의 직업은 AI 엔지니어입니다.'

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()  # ChatOpenAI 모델을 초기화합니다.

# 수도를 묻는 질문에 대한 체인을 정의합니다.
capital_chain = (
    ChatPromptTemplate.from_template("{country} 의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

# 면적을 묻는 질문에 대한 체인을 정의합니다.
area_chain = (
    ChatPromptTemplate.from_template("{country} 의 면적은 얼마입니까?")
    | model
    | StrOutputParser()
)

# capital_chain, area_chain 을 병렬로 실행할 수 있는 RunnableParallel 객체를 생성합니다.
map_chain = RunnableParallel(capital=capital_chain, area=area_chain)

# map_chain을 호출하여 대한민국의 수도와 면적을 묻습니다.
map_chain.invoke({"country": "대한민국"})


{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 총 면적은 약 100,363 제곱 킬로미터 입니다.'}

In [41]:
# 수도를 묻는 질문에 대한 체인을 정의합니다.
capital_chain2 = (
    ChatPromptTemplate.from_template("{country1} 의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

# 면적을 묻는 질문에 대한 체인을 정의합니다.
area_chain2 = (
    ChatPromptTemplate.from_template("{country2} 의 면적은 얼마입니까?")
    | model
    | StrOutputParser()
)

# capital_chain, area_chain 을 병렬로 실행할 수 있는 RunnableParallel 객체를 생성합니다.
map_chain2 = RunnableParallel(capital=capital_chain2, area=area_chain2)

# map_chain을 호출합니다. 이때 각각의 key에 대한 value를 전달합니다.
map_chain2.invoke({"country1": "대한민국", "country2": "미국"})


{'capital': '대한민국의 수도는 서울입니다.', 'area': '미국의 면적은 약 9,833,520 km² 입니다.'}

In [42]:
%%timeit

# 면저을 묻는 체인을 호출하고 실행 시간을 측정합니다.
area_chain.invoke({"country": "대한민국"})


649 ms ± 62.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit

# 수도를 묻는 체인을 호출하고 실행 시간을 측정합니다.
capital_chain.invoke({"country": "대한민국"})


667 ms ± 144 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit

# Parallel 하게 구성된 체인을 호출하고 실행 시간을 측정합니다.
map_chain.invoke({"country": "대한민국"})


778 ms ± 165 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


동적 속성 지정(configurable_fields, configurable_alternatives)

In [46]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4.1")

model.invoke("대한민국의 수도는 어디야?").__dict__


{'content': '대한민국의 수도는 **서울특별시**입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 13,
   'prompt_tokens': 15,
   'total_tokens': 28,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4.1-2025-04-14',
  'system_fingerprint': 'fp_daf5fcc80a',
  'id': 'chatcmpl-C6RvAfOy5k7IOGJzBePnQq1lCrQ7u',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--52ea9e9d-7cf4-4c27-ac0e-e1bc468af941-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 13,
  'total_tokens': 28,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [47]:
model = ChatOpenAI(temperature=0).configurable_fields(
    model_name=ConfigurableField(  # model_name 은 원래 ChatOpenAI 의 필드입니다.
        id="gpt_version",  # model_name 의 id 를 설정합니다.
        name="Version of GPT",  # model_name 의 이름을 설정합니다.
        # model_name 의 설명을 설정합니다.
        description="Official model name of GPTs. ex) gpt-4o, gpt-4o-mini",
    )
)


In [48]:
model.invoke(
    "대한민국의 수도는 어디야?",
    # gpt_version 을 gpt-4.1 로 설정합니다.
    config={"configurable": {"gpt_version": "gpt-4.1"}},
).__dict__


{'content': '대한민국의 수도는 **서울특별시**입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 13,
   'prompt_tokens': 15,
   'total_tokens': 28,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4.1-2025-04-14',
  'system_fingerprint': 'fp_daf5fcc80a',
  'id': 'chatcmpl-C6RvlOwPBzwwXt6cmRgkVX0lSFetl',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--9c5f176a-4a8c-43b7-8980-e286b2c2fac7-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 13,
  'total_tokens': 28,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [49]:
model.invoke(
    # gpt_version 을 gpt-4o-mini 로 설정합니다.
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4.1-mini"}},
).__dict__


{'content': '대한민국의 수도는 서울입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 8,
   'prompt_tokens': 15,
   'total_tokens': 23,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4.1-mini-2025-04-14',
  'system_fingerprint': 'fp_37c45ea698',
  'id': 'chatcmpl-C6RvyUUACTOMZWzZ028IO6YSLORPs',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--0a018220-3786-4bc0-b55d-9f62b250d2d9-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 8,
  'total_tokens': 23,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [50]:
model.with_config(configurable={"gpt_version": "gpt-4.1"}).invoke(
    "대한민국의 수도는 어디야?"
).__dict__


{'content': '대한민국의 수도는 **서울특별시**입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 13,
   'prompt_tokens': 15,
   'total_tokens': 28,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4.1-2025-04-14',
  'system_fingerprint': 'fp_3502f4eb73',
  'id': 'chatcmpl-C6RwAkKAN4zpz8wIdEk2dUmIyzNGz',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--df302907-032c-40e1-a6b9-8c843420cc95-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 15,
  'output_tokens': 13,
  'total_tokens': 28,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [51]:
# 템플릿에서 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{x} 보다 큰 위의 난수를 선택합니다.")
chain = (
    prompt | model
)  # 프롬프트와 모델을 연결하여 체인을 생성합니다. 프롬프트의 출력이 모델의 입력으로 전달됩니다.


In [52]:
chain.invoke({"x": 0}).__dict__  # 체인을 호출하고 입력 변수 "x"에 0을 전달합니다.


{'content': '5',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 1,
   'prompt_tokens': 21,
   'total_tokens': 22,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'id': 'chatcmpl-C6RwZF80lcnPbVDSxANaLUfMHybih',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--e4674e0c-acb5-421a-9373-a30cf72abd73-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 21,
  'output_tokens': 1,
  'total_tokens': 22,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [53]:
# 체인을 호출할 때 설정을 지정하여 체인을 호출할 수 있습니다.
chain.with_config(configurable={"gpt_version": "gpt-5"}).invoke({"x": 0}).__dict__


{'content': '위에 언급된 난수 목록을 제가 볼 수 없습니다. 숫자 목록을 붙여넣어 주시면 0보다 큰 값만 골라서 드릴게요.\n\n직접 하시려면 예:\n- Python: [x for x in nums if x > 0]\n- Excel/Google Sheets: =FILTER(A:A, A:A>0) 또는 =IF(A1>0, A1, "")\n- R: nums[nums > 0]\n- SQL: SELECT value FROM your_table WHERE value > 0;\n\n부동소수점에서 거의 0인 값 제외가 필요하면 예: [x for x in nums if abs(x) > 1e-9]\n\n목록을 주시거나 원하는 형식(정렬, 중복 제거 등)을 알려주세요.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 818,
   'prompt_tokens': 16,
   'total_tokens': 834,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 640,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-5-2025-08-07',
  'system_fingerprint': None,
  'id': 'chatcmpl-C6RwnlXq3gPiioAboHug77QzBYUY3',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--9df17445-

In [54]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("teddynote/rag-prompt-korean").configurable_fields(
    # 소유자 저장소 커밋을 설정하는 ConfigurableField
    owner_repo_commit=ConfigurableField(
        # 필드의 ID
        id="hub_commit",
        # 필드의 이름
        name="Hub Commit",
        # 필드에 대한 설명
        description="Korean RAG prompt by teddynote",
    )
)
prompt


RunnableConfigurableFields(default=HubRunnable(bound=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'rag-prompt-korean', 'lc_hub_commit_hash': '4214de29c72c82d13121c710b0ee7021d31ad088f0eb1d6ddebcaaeab46b1f8b'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='#Question: \n{questi

In [55]:
# prompt 객체의 invoke 메서드를 호출하여 "question"과 "context" 매개변수를 전달합니다.
prompt.invoke({"question": "Hello", "context": "World"}).messages


[SystemMessage(content="당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='#Question: \nHello \n\n#Context: \nWorld \n\n#Answer:', additional_kwargs={}, response_metadata={})]

In [56]:
prompt.with_config(
    # hub_commit 을 teddynote/simple-summary-korean 으로 설정합니다.
    configurable={"hub_commit": "teddynote/simple-summary-korean"}
).invoke({"context": "Hello"})


ChatPromptValue(messages=[HumanMessage(content='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: Hello\n\nSUMMARY:', additional_kwargs={}, response_metadata={})])

In [64]:
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(
    temperature=0, model="claude-3-7-sonnet-latest"
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI()`와 동일합니다.
    openai=ChatOpenAI(model="gpt-4.1-mini"),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4o=ChatOpenAI(model="gpt-4.1"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm


In [66]:
# Anthropic을 기본으로 호출합니다.
chain.invoke({"topic": "아이브"}).__dict__


{'content': '# 아이브(IVE) 소개\n\n아이브(IVE)는 2021년 12월 1일 스타쉽 엔터테인먼트에서 데뷔한 6인조 K-pop 걸그룹입니다. \n\n## 멤버 구성\n- 안유진 (리더)\n- 가을\n- 레이\n- 장원영\n- 리즈\n- 이서\n\n## 주요 특징\n- 데뷔곡 "ELEVEN"으로 큰 인기를 얻었으며, 이후 "LOVE DIVE", "After LIKE" 등의 히트곡을 발표했습니다.\n- 신인 그룹임에도 불구하고 \'몬스터 루키\'라 불리며 데뷔 초부터 높은 인기를 얻었습니다.\n- 장원영과 안유진은 Mnet의 오디션 프로그램 \'프로듀스 48\'을 통해 결성된 프로젝트 그룹 IZ*ONE 출신입니다.\n- 독특한 콘셉트와 퍼포먼스로 K-pop 시장에서 강한 존재감을 보여주고 있습니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_01CM9WZ1tk7d5oHusfSJaQFz',
  'model': 'claude-3-7-sonnet-20250219',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0,
    'ephemeral_5m_input_tokens': 0},
   'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 28,
   'output_tokens': 367,
   'server_tool_use': None,
   'service_tier': 'standard'},
  'model_name': 'claude-3-7-sonnet-20250219'},
 'type': 'ai',
 'name': None,
 'id': 'run--37cf9d0f-2a32-463e-ac82-6ada0728c7fd-0',
 'ex

In [67]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "아이브"}).__dict__


{'content': '아이브(IVE)는 대한민국의 6인조 걸그룹으로, 스타쉽 엔터테인먼트 소속입니다. 2021년에 데뷔했으며, 멤버들은 안유진, 가을, 레이, 이서, 원영, 미연으로 구성되어 있습니다. 아이브는 데뷔곡 "러브 다이브 (LOVE DIVE)"로 큰 인기를 끌었고, 세련된 콘셉트와 뛰어난 퍼포먼스로 주목받고 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 112,
   'prompt_tokens': 17,
   'total_tokens': 129,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4.1-mini-2025-04-14',
  'system_fingerprint': 'fp_37c45ea698',
  'id': 'chatcmpl-C6S7TSBCvKlaaQSF5lSCEAmRdvlyv',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--74f64236-11a9-4ca3-bb64-84fa64927084-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 17,
  'output_tokens': 112,
  'total_t

In [68]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "gpt4o"}).invoke({"topic": "아이브"}).__dict__


{'content': '아이브(IVE)는 대한민국의 6인조 걸그룹으로, 스타쉽 엔터테인먼트 소속입니다. 2021년 12월 1일에 데뷔했으며, 멤버는 유진, 가을, 레이, 원영, 리즈, 이서로 구성되어 있습니다. 데뷔곡 "ELEVEN"을 시작으로 "LOVE DIVE", "After LIKE", "I AM" 등 여러 히트곡을 발표하며 큰 인기를 얻고 있습니다. 세련된 음악과 퍼포먼스, 화려한 비주얼로 국내외에서 많은 사랑을 받고 있는 대표적인 4세대 K-POP 걸그룹입니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 150,
   'prompt_tokens': 17,
   'total_tokens': 167,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4.1-2025-04-14',
  'system_fingerprint': 'fp_daf5fcc80a',
  'id': 'chatcmpl-C6S7eBh29vtznO3HCJehmjMHosHvw',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--7856ee59-6a65-4334-8e6c-c4254363c795-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],

In [69]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "anthropic"}).invoke(
    {"topic": "아이브"}
).__dict__


{'content': '# 아이브(IVE) 소개\n\n아이브(IVE)는 2021년 12월 1일 스타쉽 엔터테인먼트에서 데뷔한 6인조 K-pop 걸그룹입니다. 멤버는 안유진, 가을, 레이, 장원영, 리즈, 이서로 구성되어 있습니다.\n\n데뷔 싱글 "ELEVEN"을 시작으로 "LOVE DIVE", "After LIKE" 등의 히트곡을 발표하며 \'4세대 대표 걸그룹\'으로 자리매김했습니다. 특히 신인 그룹임에도 불구하고 데뷔 초부터 음원차트와 음악방송에서 좋은 성적을 거두며 \'몬스터 루키\'라는 별명을 얻었습니다.\n\n아이브는 독특한 콘셉트와 세련된 음악 스타일로 \'아이브만의 색깔\'을 구축하며 국내외 많은 팬들의 사랑을 받고 있습니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_01T2M7tDQg2dvxSJETxS7XTX',
  'model': 'claude-3-7-sonnet-20250219',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0,
    'ephemeral_5m_input_tokens': 0},
   'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 28,
   'output_tokens': 341,
   'server_tool_use': None,
   'service_tier': 'standard'},
  'model_name': 'claude-3-7-sonnet-20250219'},
 'type': 'ai',
 'name': None,
 'id': 'run--19204968-2e99-4dfe-94a1-d5b7c38e03d6-0',
 'example': False,
 'tool_calls': [],
 'inva

In [70]:
# 언어 모델을 초기화하고 temperature를 0으로 설정합니다.
llm = ChatOpenAI(temperature=0)

prompt = PromptTemplate.from_template(
    "{country} 의 수도는 어디야?"  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="capital",
    # 'area'이라는 새로운 옵션을 추가합니다.
    area=PromptTemplate.from_template("{country} 의 면적은 얼마야?"),
    # 'population'이라는 새로운 옵션을 추가합니다.
    population=PromptTemplate.from_template("{country} 의 인구는 얼마야?"),
    # 'eng'이라는 새로운 옵션을 추가합니다.
    eng=PromptTemplate.from_template("{input} 을 영어로 번역해주세요."),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

# 프롬프트와 언어 모델을 연결하여 체인을 생성합니다.
chain = prompt | llm


In [71]:
# config 변경 없이 체인을 호출합니다.
chain.invoke({"country": "우간다"})


AIMessage(content='우간다의 수도는 카람보입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 20, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6S8Gkzq9ybuLRcPHbt93kVEOkFE1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0de00787-e76c-4ff3-9f72-914922a2b30e-0', usage_metadata={'input_tokens': 20, 'output_tokens': 14, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [72]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "area"}).invoke({"country": "인도네시아"})


AIMessage(content='인도네시아의 면적은 약 1,904,569 제곱 킬로미터입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 25, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6S8RRdc0NUrxuwtVK3r2unsLfpYC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--426d9f26-e0ef-4081-9d2a-4be5ef07d093-0', usage_metadata={'input_tokens': 25, 'output_tokens': 29, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [73]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "population"}).invoke({"country": "미얀마"})


AIMessage(content='2021년 기준으로 미얀마의 인구는 약 5,500만 명입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 23, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6S8fS2ML7VNsr7odIo8YNbRFQZQM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6ec13a2c-086b-46ea-91b6-cecc06179180-0', usage_metadata={'input_tokens': 23, 'output_tokens': 30, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [74]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "eng"}).invoke({"input": "과일은 수박이 최고지"})


AIMessage(content='Fruits are the best, watermelon is the best.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 29, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6S8t6mR8PvTXhQd9esrbhBIcsYGK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--005d8f27-08f8-4d12-87d5-5b67299b21a5-0', usage_metadata={'input_tokens': 29, 'output_tokens': 12, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [75]:
llm = ChatAnthropic(
    temperature=0, model="claude-3-7-sonnet-latest"
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4o-mini")`와 동일합니다.
    openai=ChatOpenAI(model="gpt-4.1-mini"),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4o")`와 동일합니다.
    gpt4=ChatOpenAI(model="gpt-4.1"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

prompt = PromptTemplate.from_template(
    "{company} 에 대해서 20자 이내로 설명해 줘."  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="description",
    # 'founder'이라는 새로운 옵션을 추가합니다.
    founder=PromptTemplate.from_template("{company} 의 창립자는 누구인가요?"),
    # 'competitor'이라는 새로운 옵션을 추가합니다.
    competitor=PromptTemplate.from_template("{company} 의 경쟁사는 누구인가요?"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
chain = prompt | llm


In [76]:
# with_config 로 설정 값을 지정하여 구성할 수 있습니다.
chain.with_config(configurable={"prompt": "founder", "llm": "openai"}).invoke(
    # 사용자가 제공한 회사에 대한 처리를 요청합니다.
    {"company": "애플"}
).__dict__


{'content': '애플(Apple Inc.)의 창립자는 스티브 잡스(Steve Jobs), 스티브 워즈니악(Steve Wozniak), 그리고 론 웨인(Ron Wayne)입니다. 이들은 1976년에 애플을 공동 설립하였습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 60,
   'prompt_tokens': 17,
   'total_tokens': 77,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4.1-mini-2025-04-14',
  'system_fingerprint': 'fp_3f58d112f7',
  'id': 'chatcmpl-C6S9WlclajfKgKtH8gkRlweBxYxX7',
  'service_tier': 'default',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--fed00af7-4332-4f08-ba1f-4df751c63b5f-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 17,
  'output_tokens': 60,
  'total_tokens': 77,
  'input_token_details': {'audio': 0, 'cache_re

In [77]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"llm": "anthropic"}).invoke(
    {"company": "애플"}
).__dict__


{'content': '애플은 아이폰, 맥북 등을 만드는 세계적인 기술 기업입니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_013BXeGPc8xRSjDfxoCdSuQ5',
  'model': 'claude-3-7-sonnet-20250219',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0,
    'ephemeral_5m_input_tokens': 0},
   'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 31,
   'output_tokens': 39,
   'server_tool_use': None,
   'service_tier': 'standard'},
  'model_name': 'claude-3-7-sonnet-20250219'},
 'type': 'ai',
 'name': None,
 'id': 'run--eefdc015-374f-45cd-9897-6d2d06842f48-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 31,
  'output_tokens': 39,
  'total_tokens': 70,
  'input_token_details': {'cache_read': 0,
   'cache_creation': 0,
   'ephemeral_5m_input_tokens': 0,
   'ephemeral_1h_input_tokens': 0}}}

In [78]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"prompt": "competitor"}).invoke(
    {"company": "스페이스엑스"}
).__dict__


{'content': '스페이스엑스(SpaceX)의 주요 경쟁사로는 다음과 같은 기업들이 있습니다:\n\n1. 블루 오리진(Blue Origin) - 제프 베조스가 설립한 우주 기업\n2. 유나이티드 론치 얼라이언스(ULA) - 보잉과 록히드 마틴의 합작 회사\n3. 록히드 마틴(Lockheed Martin)\n4. 보잉(Boeing)\n5. 노스롭 그루먼(Northrop Grumman)\n6. 로켓랩(Rocket Lab)\n7. 버진 갤럭틱(Virgin Galactic)과 버진 오빗(Virgin Orbit)\n8. 아리안스페이스(Arianespace) - 유럽의 우주 발사체 회사\n\n이들 기업은 위성 발사, 우주 관광, 화물 운송, 유인 우주 비행 등 다양한 우주 산업 분야에서 스페이스엑스와 경쟁하고 있습니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_0149sJHzXhTYMdtQGzLyc4NX',
  'model': 'claude-3-7-sonnet-20250219',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0,
    'ephemeral_5m_input_tokens': 0},
   'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 31,
   'output_tokens': 328,
   'server_tool_use': None,
   'service_tier': 'standard'},
  'model_name': 'claude-3-7-sonnet-20250219'},
 'type': 'ai',
 'name': None,
 'id': 'run--2f0093e3-4756-4268-a232-cf9da50abeae-0',
 '

In [79]:
# 하나만 구성하려는 경우
chain.invoke({"company": "마이크로소프트"}).__dict__


{'content': '마이크로소프트는 세계적인 소프트웨어 기업으로 윈도우와 오피스를 개발합니다.',
 'additional_kwargs': {},
 'response_metadata': {'id': 'msg_01JR1LrcwhwgZTALCTFM4e7f',
  'model': 'claude-3-7-sonnet-20250219',
  'stop_reason': 'end_turn',
  'stop_sequence': None,
  'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0,
    'ephemeral_5m_input_tokens': 0},
   'cache_creation_input_tokens': 0,
   'cache_read_input_tokens': 0,
   'input_tokens': 36,
   'output_tokens': 47,
   'server_tool_use': None,
   'service_tier': 'standard'},
  'model_name': 'claude-3-7-sonnet-20250219'},
 'type': 'ai',
 'name': None,
 'id': 'run--fe4ceac9-0804-4f19-9c87-2fa39a6c59ab-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 36,
  'output_tokens': 47,
  'total_tokens': 83,
  'input_token_details': {'cache_read': 0,
   'cache_creation': 0,
   'ephemeral_5m_input_tokens': 0,
   'ephemeral_1h_input_tokens': 0}}}

In [80]:
# with_config 로 설정을 변경하여 생성한 체인을 별도의 변수에 저장합니다.
gpt4_competitor_chain = chain.with_config(
    configurable={"llm": "gpt4", "prompt": "competitor"}
)


In [81]:
# 체인을 호출합니다.
gpt4_competitor_chain.invoke({"company": "테슬라"}).__dict__


{'content': '테슬라(Tesla)의 주요 경쟁사는 전통적인 자동차 제조사와 신생 전기차 업체로 나눌 수 있습니다. 대표적인 경쟁사는 다음과 같습니다:\n\n### 1. 전통 자동차 회사(전기차 부문 강화 중)\n- **폭스바겐(VW Group)**: ID 시리즈 등 전기차 라인업 확대 중\n- **GM(제너럴 모터스)**: 쉐보레 볼트, GMC 허머 EV 등\n- **포드(Ford)**: 머스탱 마하-E, F-150 라이트닝 등\n- **현대기아자동차(현대차그룹)**: 아이오닉 시리즈, EV6 등\n- **메르세데스-벤츠(다임러 그룹)**: EQ 시리즈\n- **BMW**: i 시리즈 등\n- **토요타**: bZ 시리즈 등\n\n### 2. 신생 전기차 전문기업\n- **BYD(비야디, 중국)**: 중국 최대 전기차 기업이자 세계 전기차 시장 점유율 1~2위를 다투는 기업\n- **리오토(Li Auto, 중국)**\n- **샤오펑(Xpeng, 중국)**\n- **니오(NIO, 중국)**\n- **루시드(Lucid Motors, 미국)**\n- **리비안(Rivian, 미국)**\n- **폴스타(Polestar, 스웨덴/중국)**\n\n### 3. 소프트웨어·자율주행 분야의 경쟁사\n- **구글(웨이모, Waymo)**\n- **애플(Apple Car, 루머 단계)**\n- **GM 크루즈(Cruise)**\n\n테슬라는 전기차(EV), 자율주행, 배터리 기술 등 여러 분야에서 리더십을 확보하고 있지만, 점점 더 많은 기업들이 빠르게 추격하고 있습니다. 특히 중국계 신생기업이나 전통 완성차 업체들의 전동화 전환이 가속화되며 경쟁이 격화되고 있습니다.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 455,
   'prompt_tokens': 17,
   'total_tokens': 472,
   'completion_